In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import numpy as np

関数動作確認用の関数の定義

In [6]:
def status(feature):

    print ('Processing',feature,': ok')

テストデータとトレーニングデータの統合

In [7]:
def get_combined_data():
    # トレーニングデータの読み込み
    train = pd.read_csv('../../Training_test_data/train.csv')
    
    # テストデータの読み込み
    test = pd.read_csv('../../Training_test_data/test.csv')

    # トレーニングデータから目的変数を除去 
    targets = train.Survived
    train.drop('Survived',1,inplace=True)
    

    # トレーニングデータとテストデータを結合
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop('index',inplace=True,axis=1)
    
    return combined

In [8]:
combined = get_combined_data()

MrかMrsか等の判断を行い、Combinedに"Title"行を追加する関数を定義

In [9]:
def get_titles():

    global combined
    
    # Title列に敬称、役職名とかを入れる
    combined['Title'] = combined['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    
    # 敬称、役職名をまとめるために辞書を用意
    Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"

                        }
    
    # 役職名とかをまとめる
    combined['Title'] = combined.Title.map(Title_Dictionary)

In [10]:
get_titles()

In [11]:
combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


欠損値の量を示すlack列の追加

In [13]:
combined_T = combined.T
lack=np.array(len(combined_T)-combined_T.count())
combined["lack"] = lack

年齢補完の調査

In [14]:
grouped = combined.groupby(['Sex','Pclass','Title'])
grouped.median()

PassengerId   Age  SibSp  Parch      Fare  lack
Sex    Pclass Title                                                   
female 1      Miss           529.5  30.0    0.0    0.0   99.9625   0.0
              Mrs            853.5  45.0    1.0    0.0   78.1125   0.0
              Officer        797.0  49.0    0.0    0.0   25.9292   0.0
              Royalty        760.0  39.0    0.0    0.0   86.5000   0.0
       2      Miss           606.5  20.0    0.0    0.0   20.2500   1.0
              Mrs            533.0  30.0    1.0    0.0   26.0000   1.0
       3      Miss           603.5  18.0    0.0    0.0    8.0500   1.0
              Mrs            668.5  31.0    1.0    1.0   15.5000   1.0
male   1      Master         803.0   6.0    1.0    2.0  134.5000   0.0
              Mr             634.0  41.5    0.0    0.0   47.1000   0.0
              Officer        678.0  52.0    0.0    0.0   37.5500   0.0
              Royalty        600.0  40.0    0.0    0.0   27.7208   1.0
       2      Master         550.0   2.0    1.0    1.0   26.0000   1.0
              Mr             723.5  30.0    0.0    0.0   13.0000   1.0
              Officer        513.0  41.5    0.0    0.0   13.0000   1.0
       3      Master         789.0   6.0    3.0    1.0   22.3583   1.0
              Mr             640.5  26.0    0.0    0.0    7.8958   1.0

Sex, Pclass, Titleを考慮した補完関数を定義

In [15]:
def process_age():
    
    global combined
    
    # 年齢を埋める関数
    
    def fillAges(row):
        if row['Sex']=='female' and row['Pclass'] == 1:
            if row['Title'] == 'Miss':
                return 30
            elif row['Title'] == 'Mrs':
                return 45
            elif row['Title'] == 'Officer':
                return 49
            elif row['Title'] == 'Royalty':
                return 39

        elif row['Sex']=='female' and row['Pclass'] == 2:
            if row['Title'] == 'Miss':
                return 20
            elif row['Title'] == 'Mrs':
                return 30

        elif row['Sex']=='female' and row['Pclass'] == 3:
            if row['Title'] == 'Miss':
                return 18
            elif row['Title'] == 'Mrs':
                return 31

        elif row['Sex']=='male' and row['Pclass'] == 1:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 41.5
            elif row['Title'] == 'Officer':
                return 52
            elif row['Title'] == 'Royalty':
                return 40

        elif row['Sex']=='male' and row['Pclass'] == 2:
            if row['Title'] == 'Master':
                return 2
            elif row['Title'] == 'Mr':
                return 30
            elif row['Title'] == 'Officer':
                return 41.5

        elif row['Sex']=='male' and row['Pclass'] == 3:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 26
    
    combined.Age = combined.apply(lambda r : fillAges(r) if np.isnan(r['Age']) else r['Age'], axis=1)
    status("age")

In [16]:
process_age()

Processing age : ok


現行のデータ量調査

In [17]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
lack           1309 non-null int64
dtypes: float64(2), int64(5), object(6)
memory usage: 133.0+ KB


Nameを削除し、Titleをダミー変数に変換する関数を定義

In [18]:
def process_names():
    
    global combined
    # Nameを削除
    combined.drop('Name',axis=1,inplace=True)
    
    # ダミー変数を追加
    titles_dummies = pd.get_dummies(combined['Title'],prefix='Title')
    combined = pd.concat([combined,titles_dummies],axis=1)
    
    # Titleを削除
    combined.drop('Title',axis=1,inplace=True)
    
    status('names')

In [19]:
process_names()

Processing names : ok


In [20]:
combined.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,lack,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,1,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0.0,0.0,1.0,0.0,0.0,0.0
1,2,1,female,38.0,1,0,PC 17599,71.2833,C85,C,0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0.0,1.0,0.0,0.0,0.0,0.0
3,4,1,female,35.0,1,0,113803,53.1000,C123,S,0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,3,male,35.0,0,0,373450,8.0500,NaN,S,1,0.0,0.0,1.0,0.0,0.0,0.0


1個のFareの欠損値を平均値補完する関数を定義

In [21]:
def process_fares():
    
    global combined
    # 補完
    combined.Fare.fillna(combined.Fare.mean(),inplace=True)
    
    status('fare')

In [22]:
process_fares()

Processing fare : ok


2個のembarkedの欠損値をSで埋め、ダミー変数化を行う関数を定義

In [23]:
def process_embarked():
    
    global combined
    # 補完
    combined.Embarked.fillna('S',inplace=True)
    
    # ダミーエンコーディング
    embarked_dummies = pd.get_dummies(combined['Embarked'],prefix='Embarked')
    combined = pd.concat([combined,embarked_dummies],axis=1)
    combined.drop('Embarked',axis=1,inplace=True)
    
    status('embarked')

In [24]:
process_embarked()

Processing embarked : ok


cabinの欠損値をuで埋め、cabinをダミー変数にするにする関数を定義

In [25]:
def process_cabin():
    
    global combined
    
    # 欠損値をUで埋める
    combined.Cabin.fillna('U',inplace=True)
    
    # Cabinを最初の文字だけにする
    combined['Cabin'] = combined['Cabin'].map(lambda c : c[0])
    
    # ダミーエンコーディング
    cabin_dummies = pd.get_dummies(combined['Cabin'],prefix='Cabin')
    
    combined = pd.concat([combined,cabin_dummies],axis=1)
    
    combined.drop('Cabin',axis=1,inplace=True)
    
    status('cabin')

In [26]:
process_cabin()

Processing cabin : ok


In [27]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 27 columns):
PassengerId      1309 non-null int64
Pclass           1309 non-null int64
Sex              1309 non-null object
Age              1309 non-null float64
SibSp            1309 non-null int64
Parch            1309 non-null int64
Ticket           1309 non-null object
Fare             1309 non-null float64
lack             1309 non-null int64
Title_Master     1309 non-null float64
Title_Miss       1309 non-null float64
Title_Mr         1309 non-null float64
Title_Mrs        1309 non-null float64
Title_Officer    1309 non-null float64
Title_Royalty    1309 non-null float64
Embarked_C       1309 non-null float64
Embarked_Q       1309 non-null float64
Embarked_S       1309 non-null float64
Cabin_A          1309 non-null float64
Cabin_B          1309 non-null float64
Cabin_C          1309 non-null float64
Cabin_D          1309 non-null float64
Cabin_E          1309 non-null float64
Cabin_F 

性別を0, 1に変更する関数を定義

In [28]:
def process_sex():
    
    global combined
    
    combined['Sex'] = combined['Sex'].map({'male':1,'female':0})
    
    status('sex')

In [29]:
process_sex()

Processing sex : ok


In [ ]:
pclassをダミー変数にする関数を定義

In [30]:
def process_pclass():
    
    global combined
    # pclassをダミー変数化
    pclass_dummies = pd.get_dummies(combined['Pclass'],prefix="Pclass")
    
    # ダミー変数の追加
    combined = pd.concat([combined,pclass_dummies],axis=1)
    
    # pclassの削除
    
    combined.drop('Pclass',axis=1,inplace=True)
    
    status('pclass')

In [31]:
process_pclass()

Processing pclass : ok


チケットのprefixを調べて、ダミー変数化する関数を定義

In [32]:
def process_ticket():
    
    global combined
    
    def cleanTicket(ticket):
        ticket = ticket.replace('.','')
        ticket = ticket.replace('/','')
        ticket = ticket.split()
        ticket = map(lambda t : t.strip() , ticket)
        ticket = list(filter(lambda t : not t.isdigit(), ticket))
        if len(ticket) > 0:
            return ticket[0]
        else: 
            return 'XXX'
    

    # ダミー変数化

    combined['Ticket'] = combined['Ticket'].map(cleanTicket)
    tickets_dummies = pd.get_dummies(combined['Ticket'],prefix='Ticket')
    combined = pd.concat([combined, tickets_dummies],axis=1)
    combined.drop('Ticket',inplace=True,axis=1)

    status('ticket')

In [33]:
process_ticket()

Processing ticket : ok


ファミリーの数を3パターンに分類する関数を定義

In [34]:
def process_family():
    
    global combined
    # ファミリーの大きさを示す特徴量を追加
    combined['FamilySize'] = combined['Parch'] + combined['SibSp'] + 1
    
    # 人数によって3パターンに分類
    combined['Singleton'] = combined['FamilySize'].map(lambda s : 1 if s == 1 else 0)
    combined['SmallFamily'] = combined['FamilySize'].map(lambda s : 1 if 2<=s<=4 else 0)
    combined['LargeFamily'] = combined['FamilySize'].map(lambda s : 1 if 5<=s else 0)
    
    status('family')

In [35]:
process_family()

Processing family : ok


In [36]:
combined.head()

,PassengerId,Sex,Age,SibSp,Parch,Fare,lack,Title_Master,Title_Miss,Title_Mr,...,Ticket_STONO2,Ticket_STONOQ,Ticket_SWPP,Ticket_WC,Ticket_WEP,Ticket_XXX,FamilySize,Singleton,SmallFamily,LargeFamily
0,1,1,22.0,1,0,7.2500,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0,1,0
1,2,0,38.0,1,0,71.2833,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0,1,0
2,3,0,26.0,0,0,7.9250,1,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0
3,4,0,35.0,1,0,53.1000,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2,0,1,0
4,5,1,35.0,0,0,8.0500,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0,0


In [37]:
combined.to_csv("../data/combined.csv", index=False)